# Domain Cells

The Dcel pattern defines a _domain_ as a coupled data set and storage location [1][2].

The Dcel encapsulates a domain in a single, cell-like object with a generized IO interface.

The generalized API allows differently typed domains to be recursively chained and nested in a way that a path walker can easily traverse. [3]

Also, in a spreadsheet-cell-like fashion, Dcels may contain a formula to generate their value/dataset. The formula may refer to other Dcels, allowing dependency chains which are refreshed when needed.

The Dcel uses external modules to provide the formula function and storage service. This enables the Dcel to enclose different data and storage types.

### Executable Dcels

Dcels may contain an executable. It is not the responsibility of the Dcel class to run such.

---
1. Data sets of any size and type at a specific storage location.
2. Dcels can be overlaid to provide multiple storage locations for a single data set.
2. File paths can refer to locations within files regardless of filetype.

    


## Table of Contents

Overview
- Purpose of a Dcel
- creating a Dcel
- Lazy R Value

The Dcel

- production instruction
- service
- stream
- map
- directory
- value
- executable

API's

- pyfilesystem
- http-like
- string representation
- assoc array representation

# Overview

## Purpose of a Dcel

A 'Domain Cel' (aka 'Dcel') represents the 'domain' of a set of data.

## Creating a Dcel

A Dcel needs a combination of the following to resolve to its `value/dataset`:

- formula function with args
- service handle
- service address
- buffer value

An optional `formula function`, if present, generates the `value/dataset`. The Dcel stores the value at the `address` served by `service`. 

On access, the `service` takes the `address` and serves the `value`. The Dcel refreshes stale values via the formula function.

The `buffer value` can be set explicitly without formula or service. In this case the Dcel storage will be temporary.

### Service Address

The `address` is a base path. The service module API uses path strings to identify items within `value/dataset`. Descendants within the dataset use subpaths with the same service. A Dcel derived from a descendant uses the subpath as its `address`.


### Initializing a Dcel
    
Explicitly constructed with all inputs:

    D(formula = fn,
      args = [a],
      service = serviceobj,
      address = serviceaddr)
      
Constructed with service,address pair:

    D(service = serviceob,
      address = serviceaddr)
      
Constructed with function,args pair without a service:
      
    D(formula = fn,
      args = [a])
      
    # expected to produce a value
    # Dcel wrapper wont know
    # which service to use.
    # service functions will fail
    # but value functions succeed.    
    
Constructed with value only.

    D(value = v)
    
    # explicit buffer value
    # implied temporary memory
    
Constructed with ServiceClass,addr pair:

    D(service_class = cls,
      address = addr)
      
The ServiceClass takes a `service_address` and generates a `service` object.

The enclosing Dcel's base path `address` is `/`.


## R Value

The R Value of a dcel is evaluated lazily on access.

The Dcel may be chained from other Dcels and may be dependent on the values of other dcels. 
    
A simple value implementation would be:

    self.args = [ dcel1, dcel2 ]
    def value(self):
        return self.function(
                *self.args
                )
                
Psuedocode test:

    d = Dcel(filesvc,['.'])
    e = Dcel(lookupfn,[d,request])
    e.value = e.function(*e.args)

## Production Instructions
    
The (addr,service) pair is an example of a production instruction.

    d = D(service,[url])
    d = D(service,[string])
    d = D(service,[number])
    d = D(service,[json])

The production instruction is a repeatable instruction which can be used to rebuild or replicate the Dcel, and which is responsible for evaluating the dcel to return a value.

The python `__repr__()` method may return it?

The args to the instructions are re-evaluated on access if they contain dcels.

    d = D(func,[d2,d3])
    
vs.

    d = D(func,[1,2])
    
Which is only re-evaluated if `self._value` is not set.

#### Service vs Class

The (service,addr) pair:

Service could well be a class, and the address could be packaged in an instance of it.

For extensibility this could work if the class code is loaded into the program on the fly at runtime.

However a microservices approach to "classes" within the context of Dcels allows sandboxing. The Dcel methods delegate to a DcelService type which wraps a microservice.

### Kinds of Production Instructions

- Source
- Bind
- Find
- Transform

#### Source

    d = D(addr,service)
    
Set up dcel with an initialized service. The function of the dcel returns the service object. Calls to methods 
    
#### Bind

    d = D(targets,bindmethod)
    
#### Find

    d = D(target,lookupmethod,request)
    
The find instruction, where target should be a 'parent' dcel.
    
#### Transform

    d = D(source,transformmethod,args)
    
### Generalization

    D(target,function,args)
    
A flexible way of constructing a production instruction would be to create a dictionary or named tuple:

    target: 'this',
    function: fnobject,
    args: array or dict
    
## Service

While the Production Instruction determines the content of the Dcel, the Dcel's Service delivers it.

The Service is passed in as a pyfilesystem class, which takes the production value as input and generates an open file system handle.

    service = serviceclass(value)
    
### Service vs Production Instruction

Production and service.

    a = D(noop,
          stringdata,
          stringserver)
          
    a = D(noop,
          pathstr,
          fileserver)
          
    a = D(fileserver, # won't work
          pathstr,
          fileserver) # won't work
          
The producer and fileserver need to be different. The producer creates an object which is used by the server.

    d1 = D(noop,[string1],
           stringserver)
    d2 = D(noop,[string2],
           stringserver)
    a = D(concat,
          [dcel1,dcel2],
          stringserver)
            
    out = a.readtext()
    i = a.getinfo()
    child = a[key]
    
### Reusing Open Services

Parent Dcel

    prodfn(args) -> servableob
    -> serviceclass()
    -> serviceob, baseaddr

Child Dcel

    serviceob.exists(childaddr)
    -> sameserviceob, childaddr

## Value and Executable

The primary value from the production instruction is cached before being passed to the service.

The "value" of a Dcel can be obtained from the output of `getinfo()` if the service supports the `dcel` namespace.

    i = d.getinfo('.',namespaces=['dcel'])
    val = i.raw['dcel']['value']

Any object, including callables, can be passed from the service. So functions as well as data can be delivered by the service.


## Map and Slices (Fragments)

The Dcel can have a `_map` property which contains `fragments` or `slices` which have been altered.

### Slices (Fragments)

    a = Dcel("grand opening")
    b = a[6:13] # maps 'opening'
    b.value = "poobah" # replace 'opening'
    print(a)
    
    > grand poobah


### Map on Write




## The Dcel API

A Dcel has a FS API.

    dcel.listdir()
    dcel.readbytes()
    etc...
    
A Dcel has a directory surface.

    dcel[childkey]

A Dcel has a properties surface.

    dcel.getinfo(...)
    
A Dcel has a value.

    dcel.value
    
Dcels can be empty.

    
## API Interactions

How do different API's interact with a Dcel?

### Delegation

The Dcel provides methods which delegate to the service object. Services are `pyfilesystem` compatable classes.

### Production and API

A Dcel is essentially a production instruction with a map and a directory. The service is responsible for noticing if the Dcel state is stale.

Services provide some or all of the following:

__Service__
- data for the stream
- a map of the stream
- a directory of children
- private state

A service doesn't produce a dcel, it produces the goods that the Dcel delivers.

#### Getitem and Lookup

The Dcel getitem method tests the service for presence of the item.

Uses DcelService.lookup()

    baseaddr, addr -> itemaddr

The `lookup()` method is required to return an address of an item in such a state that the address can be used to produce a new Dcel instance, using the existing service instance.

In this way, lookups and `__getitem__()` calls on the Dcel can be chained.

### Service and Environment

The service runs inside an environment established by an APath. Services can reference other services and external resources as provided by the APath's `.cosm`.

The service object is instantiated within the .cosm environment.


### Dcel and Environment

The Dcel doesn't contain any APath content from the parent invironment, but may provide some environment from its own data. The dcel is a passive container. The APath requests a '.cosm' from the Dcel and the DcelService looks it up.





In [90]:
from fs.base import FS
from fs.info import Info
from fs.errors import NoURL, InvalidPath, ResourceNotFound, DirectoryExpected
from fs.path import normpath
from inspect import isclass, isfunction, ismethod
from urllib.parse import urlparse
from mergeinfo import mergeinfo
from ValueSvc import ValueSvc

DEFAULT_HOST = 'localhost'

class DcelReference:
    def __init__(self,dcel):
        self._dcel = dcel
        
    def __invert__(self):
        return self._dcel
    
    def __str__(self):
        return str(self._dcel)
    
class DcelIterator:
    """WIP: Don't use this yet. Dcel::__iter__() should return an iterator derived from Dcel::value"""
    def __init__(self,target):
        if target != None:
            self.target = target
            self.curpos = 0
        
    def __iter__(self):
        return self
    
    def __next__():
        raise StopIteration
        

class Dcel(FS):
    
    def __new__(cls, arg1=None, *args, **kwargs):
        if type(arg1) is Dcel:
            if 'writeable' in kwargs:
                # This allows cloning a new Dcel
                # from an old one, with a different `writeable`
                # property.
                if kwargs['writeable'] != arg1._writeable:
                    return super().__new__(cls)
            return arg1
        else:
            return super().__new__(cls)
                
    
    def __init__(self,
                 address = None,
                 service = None,
                 formula = None,
                 args = None,
                 value = None,
                 service_class = None,
                 service_args = None,
                 kwargs = None,
                 writeable = True,
                ):
        """
        Initialize a new Dcel object.
        
        If self.__inited == True, then we are unwrapping the 
        dcel from a wrapper, and no initialization is needed.
        
        These are the main pathways:
        - address only (expected to use as a value)
        - address and service
        - address and service_class
        - slice
        """
        # __inited
        try:
            """
            The Apath
            class might be unwrapping
            an Apath from a wrapper.
            """
            if self.__inited == True:
                return
        except AttributeError:
            # a new instance.
            super().__init__()
            self.__inited = True
        
        # _writable
        # Note to future self:
        # Don't set _writeable until end of initialization
        # or you are in for a city-state of pain.
        # Example: `self._writeable = writeable`
        # DO the sensible thing.
        self._writeable = True
        # And then please remember to set the actual value
        # before the method returns.
        
        # If the address is a dcel,
        # use the dcel's value
        # and save the dcel in case
        # it needs to be re-evaluated.
        
        def init_address_only(address):
            if type(address) is Dcel:
                # Clone a Dcel but
                # allow current __init__() to override 'writeable' property.
                try:
                    self.address = address.address
                except:
                    self.address = address.value
                try:
                    self.service = address.service
                except:
                    pass
            elif issubclass(type(address),(str,int,bool)):
                self.address = '.'
                self.service = ValueSvc(address)
        
        def init_address_service(address, service):
            self.address = address
            self.service = service
        
        def init_address_service_class(address,
                                       service_class,
                                       args=None,
                                       kwargs=None
                                      ):
            if (isclass(service_class)
                or isfunction(service_class)
                or ismethod(service_class)):
                self.address = "/"
                try:
                    url = urlparse(address)
                    hostname = url.netloc
                    path = url.path
                    if hostname == '':
                        hostname = path
                except:
                    hostname = DEFAULT_HOST
                if service_args:
                    self.service = service_class(address,**service_args)
                else:
                    self.service = service_class(address)
                self.service.__rooturl = address
        
        # NEW switch to init
        # - pathway: address only
        if ( address and
             not service and
             not formula and
             not args and
             not value and
             not service_class and
             not service_args and
             not kwargs
             # writeable will be processed at the end of __init__()
           ):
            init_address_only(address)
        # - pathway: address and service
        elif ( address and service and
             not formula and
             not args and
             not value and
             not service_class and
             not service_args and
             not kwargs
             # writeable will be processed at the end of __init__()
           ):
            init_address_service(address, service)
        # - pathway: address and service_class
        elif ( address and service_class and
             not service and
             not formula and
             not value
             # (optional) args
             # (optional) service_args
             # (optional) kwargs
             # writeable will be processed at the end of __init__()
           ):
            init_address_service_class(address,
                                       service_class,
                                       *(args if args else []),
                                       *(service_args if service_args else []),
                                       # args and service_args will be concatenated
                                       # if received with *args signature.
                                       **(kwargs if kwargs else {})
                                      )
                
        # Deprecate Dcel formula parameter
        # in favor of FormulaFS service_class
        if formula and args:
            self.formula = formula
            self.args    = args
            self.kwargs  = kwargs
        elif formula:
            self.formula = formula
            
        # Value
        if not value is None:
            self.value = value
            
        
        # internal stubs
        self._map = None
        self._dir = None
        
        # last word
        # (Thank you for keeping this here, as it must be.)
        self._writeable = writeable
        
    def __invert__(self):
        return DcelReference(self)
            
    def __str__(self):
        # this needs to
        # return a str version of
        # self.value
        return str(self.value)
    
    def freshen(self, a):
        if type(a) is Dcel:
            return a.value
        if type(a) is DcelReference:
            return ~a
        return a
    
    def preslice_value(self):
        _value = ""
        # value from slice
        if (hasattr(self,'address')
            and hasattr(self,'service')
            and type(self.address) is slice
            and type(self.service) is Dcel):
            if (hasattr(self,'_dirty')
                and hasattr(self,'_value')):
                _value = self._value
                return _value
            else:
                _value = str(self.service.preslice_value())[self.address]
                return _value
            
        # value from service.value
        if (hasattr(self,'service')
            and hasattr(self.service,'value')):
            return self.service.value
        
        # value from readtext
        try:
            _value = self.readtext('.')
            self._value = _value
        except:
            try:
                _value = self.readbytes('.')
                self._value = _value
            except:
                pass
        
        # value from formula
        # DEV: Depricating in favor of FormulaFS service_class.
        try:
            # the dirty flag becomes available
            # the first time self.value is set.
            if not hasattr(self,'_dirty'):
                try:
                    _args = [ self.freshen(a)
                              for a in self.args ]
                except:
                    _args = []
                try:
                    _kwargs = { k:self.freshen(v) 
                           for k,v in self.kwargs.items() }
                except:
                    _kwargs = {}
                try:
                    _value = self.formula(*_args,**_kwargs)
                    self.value = _value
                    return _value
                except:
                    pass
            else:
                # return buffer
                if hasattr(self._value):
                    return self._value
        except:
            pass

        # value from getinfo
        try:
            i = self.getinfo(namespaces=['dcel'])
            _value = i.raw['dcel']['value']
            self._value = _value
            return _value
        except:
            pass

        # value from buffer
        try:
            _value = self._value
            return _value
        except:
            pass
        return None
    
    @property
    def value(self):
        _value = self.preslice_value()

        # value from fragment map
        if not self._map is None:
            _base = _value
            if type(_base) is bytes:
                _base = _base.decode('utf-8')
            pos = 0
            _value = ""
            for ea in self._map:
                if pos < ea:
                    _value += str(_base[pos:ea])
                _fragcel = self._map[ea][1]
                if _fragcel is self:
                    _value += str(_value)
                _value += str(_fragcel)
                advancepos = self._map[ea][0]
                if advancepos is None:
                    return _value
                pos = advancepos
            _value += str(_base[pos:])
        return _value
    
    def conform_map(self):
        """Update map with new offsets."""
        pass
        
    @value.setter
    def value(self,new_value):
        """Set the value of the Dcel.
        
        The value can be stored in multiple ways.
        The value is buffered, but the buffer may
        either be backed by something or not backed
        by anything.
        
        If the value changes, there are multiple effects.
        If the Dcel is buffered without backing,
        the buffer changes and that's that. If there
        is a backing on the Dcel (the Dcel has either a service
        or a formula attribute) then the backing needs to be
        updated.
        
        Multiple backing scenarios exist: If the Dcel
        is backed by a URL, and the entire contents of the
        dcel are updated, then the entire contents can be
        flushed to the URL in a single write operation.
        
        If the Dcel is backed by another Dcel, it possibly
        represents a fragment of the other Dcel. The changed
        fragment needs to be inserted into the backing Dcel
        and then the backing Dcel is either flushed to its URL
        or inserted into its backing cell, and so forth until
        a root backing cell is flushed to its URL.
        
        Given that multiple fragment Dcels might be updated
        in a short succession, or batch updated, it could
        be more efficient to wait until several dirty fragments
        are ready before flushing the root backing cell to 
        its URL. A time window could be implemented to
        allow serial fragment writes. A batch-flush mechanism
        could be implemented to allow fragment update batches.
        
        The Dcel fragment map allows for the representation
        of the root backing to include updated fragments. The
        Dcel.readtext() function should step through the bytes
        of the filestream and insert changed fragments.
        
        """
        
        def set_dirty():
            # if this is a subsequent time
            if hasattr(self,'_dirty'):
                self._dirty = True
                
            # if this is the first time
            if not hasattr(self,'_dirty'):
                self._dirty = False
        
        if self._writeable == False:
            raise Exception('not writeable')
        
        # set buffer if contents changed
        if not hasattr(self,'_value'):
            self._value = new_value
            set_dirty()
        elif self._value != new_value:
            self._value = new_value
            set_dirty()
            
        
        # write slice "fragment"
        if (hasattr(self,"address")
            and hasattr(self,"service")
            and type(self.address) is slice
            and type(self.service) is Dcel):
            self.service.fragment_updated(self.address, self)
        
        # write to service
        try:
            self.service.writetext(self.address,
                           new_value)
            return
        except:
            pass
        
        # write to setinfo
        try:
            i = {'dcel':{'value':new_value}}
            self.setinfo(i)
            return
        except:
            pass
        
    def fragment_updated(self,frag_slice,frag_dcel):
        # TODO: Set and monitor timer to flush buffer to backing.
        if self._map is None:
            self._map = dict()
        self._map.update({frag_slice.start:(frag_slice.stop,frag_dcel)})
        if (hasattr(self,"address")
            and hasattr(self,"service")
            and type(self.address) is slice
            and type(self.service) is Dcel):
            assert(not self.service is self)
            self.service.fragment_updated(self.address,self)
                    
    def flush(self):
        if (hasattr(self,"address")
            and hasattr(self,"service")):
            address = self.address
            service = self.service
            # bubble the flush...
            if (type(address) is slice
                and type(service) is Dcel):
                service.flush()
                return
            elif (type(address) is Dcel):
                address.flush()
                return
            else:
                # I have to do this tacky shit in order to
                # put my kids to bed on time.
                # ARCHITECTURE FLAW:
                # For the future: Don't add any external methods to a
                # service derived from pyfilesystem.
                # For the band-aide kludge for DictFS dcels
                # which hold Dcel's in their leaves:
                if(hasattr(service,"lookup")):
                    target = service.lookup(address)
                    if (type(target) is Dcel):
                        if not self._map is None:
                            target.value = self.value
                        target.flush()
                        return
                    
        # FIXED 8-7-2022 - raygan
        # MulticelFS needs to flush correctly.
        # The following appears to work.

        # execute the flush
        if not self._map is None:
        # The following seems super-strange
            # however, the getter will build the value
            # based on the fragment map
            # and the setter will write to backing.
            # If there is no backing, the buck stops
            # at self._value.
            self.value = self.value
        self._dirty = False
        self._map = None
            
    # Lookup interface
    def __getitem__(self,key):
        """
        Access a child of the Dcel's
        domain.
        
        Access a slice of the Dcel's
        domain.
        
        Return object with a dcel
        interface that represents a
        child from the "directory"
        surface of this dcel.
        
        The service lookup()
        method is expected to return
        an address-object suitable
        for use with the same service
        instance.
        """
        # slice
        if type(key) is slice:
            return Dcel(address=key,service=self,writeable=self._writeable)
        # before adding a _dir,
        # does the service see this
        # base address (not the key) as a dir?
        
        # WIP Dcel address
        # convert Dcel value to string type
        if (type(key) is Dcel):
            key = str(key)
        
        # function?
        if hasattr(self, 'formula'):
            raise DirectoryExpected('Formula cell has no directory.')
        
        # indexed or iterable?
        if (not hasattr(self,'address')
           and not hasattr(self,'service')
           and (hasattr(self.value,'__getitem__'
                or hasattr(self.value,'__iter__')))):
            if key in ['/','.','']:
                return self
            return Dcel(self.value[key])

        # testing base address for dir
        if not self.service.isdir(self.address):
            pass
            #raise TypeError(self.address)

        # base address of this Dcel is a directory
        # now we can look up the key.

        if not type(self._dir) is dict:
            # self._dir is not dict
            basepath = self.address
            if basepath == '/':
                basepath = ''
            # create address ("path") of key relative to service
            if type(basepath) is str:
                path = basepath + '/' + key.lstrip('/')
            else:
                path = key.lstrip('/')
            if self.service.exists(path):
                # create Dcel's internal _dir and add child
                child = Dcel(address=path,
                             service=self.service,
                             writeable=self._writeable
                            )
                self._dir = dict()
                self._dir[key] = child
                # return the child
                return child
            else:
                raise KeyError(key)
        
        try:
            # return child if already cached
            # in Dcel's internal _dir
            return self._dir[key]
        except KeyError:
            # otherwise, add child to Dcel's _dir
            # same code as further above.
            basepath = self.address
            if basepath == '/':
                basepath = ''
            if key == None:
                path = basepath
            else:
                path = basepath + '/' + key.lstrip('/')
            if self.service.exists(path):
                # child
                child = Dcel(address=path,
                            service=self.service,
                             writeable=self._writeable
                            )
                self._dir[key] = child
                return child
            else:
                raise KeyError(key)
        except TypeError:
            raise TypeError(self._dir)
            
    def __setitem__(self,key,value):
        if self._writeable == False:
            raise Exception('not writeable')
        self[key].value = value
        
    def __iter__(self):
        """Return an iterator derived from `self.value`.
        
        If self.value is a string, just return an iterator
        of a list with that string as a single element.
        """
        if (hasattr(self.value,'__iter__')
            and self.value.__iter__ != None):
            if type(self.value) != str:
                return self.value.__iter__()
            else:
                return [self.value].__iter__()
    
        
    def keys(self):
        if hasattr(self,'service'):
            return self.listdir('.')
        if hasattr(self.value,'keys'):
            return self.value.keys()
        else:
            return None
            

    
    # Info helpers
    
    ## Absolute path.
    #  Be _very_ careful modifying this.
    #  Because there are potentially different object
    #  types which can be used for the address,
    #  _especially_ `slice` objects, we can't really modify
    #  anything but 'pathlike' objects, which have
    #  string representations.
    def abspath(self, addr=None):
        if hasattr(self,'address'):
            _self_address = self.address
        else:
            _self_address = ''
        if type(addr) is str:
            if not addr in ['/','.','']:
                if _self_address == '':
                    return _self_address + '/' + addr.lstrip('/')
                else:
                    return addr
            else:
                if hasattr(self,'address'):
                    return self.address
                return addr
        if addr != None:
            return addr
        else:
            return _self_address
    
    @property
    def hostname(self):
        try:
            return self.service.geturl('/')
        except:
            return DEFAULT_HOST
    
    def getinfo(self,
                path=None,
                namespaces=None
               ):
        path = self.abspath(path)
        info = self.service.getinfo(path,namespaces)
        # MUST allow pyfilesystem conformant subclasses
        # to omit geturl().
        try:
            rooturl = self.service.__rooturl
        except:
            rooturl = None
        if not type(rooturl) is str:
            rooturl = None
        if not rooturl is None:
            hostinfo = { 'hosts': [rooturl] }
            raw = mergeinfo(info.raw, hostinfo)
        else:
            raw = info.raw
        return Info(raw)
    
    def listdir(self,path=None):
        if hasattr(self, 'service'):
            path = self.abspath(path)
            return self.service.listdir(path)
    
    def isdir(self,path=None):
        
        if hasattr(self, 'service'):
            _path = self.abspath(path)
            return self.service.isdir(_path)
        
        _value = self.value
        if (hasattr(_value,'__getitem__')
            and not type(_value) is str):
            try:
                # Dcel will respond to __getitem__
                # so we need to see if iter() works.
                res = iter(_value)
                return True
            except:
                return False
        else:
            return False

    def openbin(self,
            path=None,
            mode='r',
            buffering=-1,
            **options):
        # Open a binary file.
        path = self.abspath(path)
        if self._writeable == False:
            mode='r'
        return self.service.openbin(
            path,
            mode,
            buffering)
    
    def readbytes(self, path=None):
        path = self.abspath(path)
        return self.service.readbytes(path)

    def readtext(self, path=None):
        path = self.abspath(path)
        return self.service.readtext(path)
    
    def setinfo(self,path,info):
        if self._writeable == False:
            raise Exception('not writeable')
        path = self.abspath(path)
        return self.service.setinfo(path,info)

    def makedir(self,*args,**kwargs):
        if self._writeable == False:
            raise Exception('not writeable')
        return self.service.makedir(*args,**kwargs)
        
    def remove(self,*args,**kwargs):
        if self._writeable == False:
            raise Exception('not writeable')
        self.service.remove(*args,**kwargs)
        
    def removedir(self,*args,**kwargs):
        if self._writeable == False:
            raise Exception('not writeable')
        self.service.removedir(*args,**kwargs)
        
    def writetext(self, path=None, contents='', encoding='utf-8',
                  errors=None, newline=''):
        if self._writeable == False:
            raise Exception('not writeable')
        path = self.abspath(path)
        self.service.writetext(path,contents,encoding,errors,newline)

    ### Additional Methods
    
    def _pathwalk(self,path):
        if path in ['/', '.', '', None]:
            return self
        try:
            seg,nextpath = path.split('/',1)
        except:
            return self[path]
        try:
            if seg in ['/', '.', '']:
                target = self
            else:
                target = self[seg]
        except:
            raise ResourceNotFound(seg)
        try:
            return target._pathwalk(nextpath)
        except:
            raise ResourceNotFound(nextpath)
        
    def path_lookup(self,path):
        try:
            return self._pathwalk(path)
        except:
            raise
    
    ### Custom Request Processing
    
    def processRequest(self,path,rq):
        if hastattr(self.service,"processRequest"):
            try:
                response = self.service.processRequest(path,rq)
            except Exception as e:
                return {'Dcel::processRequest':{'service error':str(e)}}
            return response
    
    def inspect(target):
        if not target:
            return "Target was empty. Nothing to inspect."
        if hasattr(target, "address"):
            _address = target.address
            _address_type = type(target.address)
            _abspath = target.abspath(target.address)
            _abspath_type = type(_abspath)
        else:
            _address = ''
            _address_type = 'Address attribute not present.'
            _abspath = target.abspath(None)
            _abspath_type = 'Abspath calulated from `None`.'
        if hasattr(target, "service"):
            _service = target.service
            _service_type = type(target.service)
        else:
            _service = ''
            _service_type = 'Service attribute not present.'
        # Note to self: `service_class` is an `__init__()` keword arg
        # which resolves to the `.service` attribute during `__init__()`.
        if hasattr(target,"value"):
            _value = target.value
            _value_type = type(target.value)
        else:
            _value = ''
            _value_type = 'Value not present.'
        if hasattr(target,"_map"):
            _map = target._map
            _map_type = type(target._map)
        else:
            _map = ''
            _map_type = '_map not present.'
        if hasattr(target,"_dir"):
            _dir = target._dir
            _dir_type = type(target._dir)
        else:
            _dir = ''
            _dir_type = 'Dir not present.'

        return f"""address: {_address_type}:{_address}
        abspath: {_abspath_type}:{_abspath}
        service: {_service_type}:{_service}
        value: {_value_type}:{_value}
        _map: {_map_type}:{_map}
        _dir: {_dir_type}:{_dir}
        """


In [91]:
True if (True and True) else False

True

In [83]:
# Dev Dcel_v2

class Dcelv2():
    def __init__(self, value_class, value, *args, **kwargs):
        self.value = value_class(value, *args, **kwargs)
        
    def __str__(self):
        return str(self.value)
        

In [42]:
d2 = Dcelv2(str,"hello")
d2i = Dcelv2(int,1)

In [43]:
print(d2)
print(d2i)

hello
1


In [84]:
# Test Dcel without a value
noncel = Dcel()

In [87]:
print(noncel.inspect())

address: Address attribute not present.:
        abspath: Abspath calulated from `None`.:
        service: Service attribute not present.:
        value: <class 'NoneType'>:None
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [88]:
print(noncel)

None


In [44]:
# Test ValueSvc
vcel = Dcel('hello')

In [45]:
print(vcel.inspect())

address: <class 'str'>:.
        abspath: <class 'str'>:.
        service: <class 'ValueSvc.ValueSvc'>:<ValueSvc.ValueSvc object at 0x7f4ccdb583d0>
        value: <class 'str'>:hello
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [46]:
print(vcel)

hello


In [47]:
# test open_fs
from fs import open_fs

In [48]:
fsd = Dcel('demo-files/fs',service_class=open_fs)

In [50]:
print(fsd.inspect())

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class 'fs.osfs.OSFS'>:<osfs '/home/raygan/Cosms/Dboy/Laydbug/dev/cburnfs_py/cburnfs/demo-files/fs'>
        value: <class 'NoneType'>:None
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [55]:
fsd.listdir('/')

['fruit', 'hello.txt', '@', 'test.txt']

In [72]:
print(fsd['fruit']['apples.txt'][5:8].inspect())

address: <class 'slice'>:slice(5, 8, None)
        abspath: <class 'slice'>:slice(5, 8, None)
        service: <class '__main__.Dcel'>:Some are red, some are green.
        value: <class 'str'>:are
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [7]:
listdcel = Dcel(address=['a','b','c'])

In [16]:
listclone = list(listdcel)

In [17]:
print(listclone)

['a', 'b', 'c']


In [2]:
### Test: formula Dcel, execution of

from ApathRootCosm import apathRootCosm

In [3]:
fn = apathRootCosm['bin']['hello']
fn_dcel = Dcel(formula=fn)

In [4]:
# This should print 'hello.' as a side-effect of the initial execution.
# The return value of 'hello()' is 'hello there.'
# The return value is cached.
print(fn_dcel)

# The subsequent stringification of fn_dcel uses the cached value.
# There should not be any side-effect, as the function does not execute this time.
print(fn_dcel)

hello.
hello there.
hello there.


In [5]:
# testing formulat cel with `isdir()`
fn_dcel.isdir()

False

In [6]:
fn_dcel.listdir()

In [7]:
# testing formula cel with __getitem__()

fn_dcel['@'].value

# Observation: __getitem__() executes the formula 2x.

DirectoryExpected: path 'Formula cell has no directory.' should be a directory

In [8]:
# wrapping formula cel

from DictFS import DictFS

wrap_cel = Dcel(address=fn_dcel, service_class=DictFS)

print(wrap_cel.isdir())
# Observation: Even though the cell is double-up, `isdir()` only invokes one execution.

<class 'str'>
False


In [9]:
type(wrap_cel)

__main__.Dcel

### Test: `__iter__()` preserves Dcel values of a single string.

In [10]:
str_dcel = Dcel("hello")
for each in str_dcel:
    print(each)

hello


### SETUP: test paths

In [11]:
testpaths = ['/','.','',None,'//3',slice(0,1,None)]

### TEST: `abspath()` method

In [12]:
from DictFS import DictFS
d = Dcel({'1': 'one', '2': {'du': 'dos'}}, service_class=DictFS)

In [13]:
for each in testpaths:
    print(f"{each}: {d['2']['du'].abspath(each)}")

/: /2/du
.: /2/du
: /2/du
None: /2/du
//3: //3
slice(0, 1, None): slice(0, 1, None)


### TEST: `isdir()` method

In [14]:
for each in ['1','2','/']:
    print(f"{each}: {d[each].isdir()}")

1: False
2: True
/: True


### TEST: DictFS wrapped in Dcel

In [22]:
from DictFS import DictFS

In [23]:
dict_src = {'a':{'Aye':'A'}}
dict_dcel = Dcel(dict_src)
dictfs_dcel = Dcel(address=dict_dcel, service_class=DictFS)

In [24]:
print(type(dict_dcel))
print(dict_dcel['a']['Aye'])

<class '__main__.Dcel'>
A


In [25]:
print(dict_dcel['a']['Aye'])

A


In [26]:
print(dict_dcel['a'])
print(type(dict_dcel['a']))
print(dict_dcel['a']['Aye'])
print(type(dict_dcel['a']['Aye']))

{'Aye': 'A'}
<class '__main__.Dcel'>
A
<class '__main__.Dcel'>


In [27]:
print(dictfs_dcel)

Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
{'a': {'Aye': 'A'}}


In [28]:
print(dictfs_dcel)
print(dictfs_dcel['a'])
print(type(dictfs_dcel['a']))
print(dictfs_dcel['a'].inspect())
print(dictfs_dcel.inspect())
print(dictfs_dcel['/'].inspect())


Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
{'a': {'Aye': 'A'}}
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
{'Aye': 'A'}
Dcel::_pathwalk: path type <class 'str'>
<class '__main__.Dcel'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>


### TEST: Dcel from fstab_hg parser

In [29]:
from fstab_hg import fstab_hg
from HienaMP import hiena_mp
from fs import open_fs
from DictFS import DictFS
#from Dcel import Dcel

In [ ]:
fs_dcel = Dcel('demo-files/fs',service_class=open_fs)

In [43]:
# Debug: preview first 100 bytes
print(fs_dcel['@']['etc']['fstab'][0:100])

# # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 


In [31]:
fstab_live = Dcel(formula=hiena_mp,args=[fstab_hg,~(fs_dcel['@']['etc']['fstab'])])

In [45]:
# Debug
print(fstab_live.inspect())

address: Address attribute not present.:
        abspath: Abspath calulated from `None`.:
        service: Service attribute not present.:
        value: <class 'NoneType'>:None
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [32]:
print(fstab_live['2']['file'].inspect())

DirectoryExpected: path 'Formula cell has no directory.' should be a directory

In [38]:
fstab_wrap = Dcel(address=fstab_live, service_class=DictFS)

In [39]:
print(fstab_wrap['1'].inspect())

Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>
Dcel::_pathwalk: path type <class 'str'>


KeyError: '1'

In [7]:
print(fstab_wrap['1']['file'].inspect())

<class 'str'>
<class 'str'>
<class 'str'>


KeyError: '1'

In [57]:
#### CLEANUP ####
fs_dcel.close()

### Tests: Dcel Address Types

In [113]:
## Test: Address Types
from DictFS import DictFS
from fs import open_fs

dtest_fs = Dcel('demo-files/fs', service_class=open_fs)
dtest_file = dtest_fs['fruit']['apples.txt']
dtest_file_slice = dtest_file[0:12]
dtest_file_slice_slice = dtest_file_slice[9:12]
dtest_string = Dcel("Hello, friend.")
dtest_string_slice = dtest_string[0:5]
dtest_dict = Dcel({"a":{"Aye":"A"},"b":{"Bee":"B"}})

dtest = {
    'string': dtest_string,
    'string_slice': dtest_string_slice,
    'dict': Dcel({"a":"Ayee"}, service_class=DictFS),
    'directory': dtest_fs,
    'file_slice_slice': dtest_file_slice_slice,
    'file_slice': dtest_file_slice,
    'file': dtest_file,
    'dict': dtest_dict,
}

In [114]:
## Restore values before modification
dtest_file.value = "Some are red, some are green."
dtest_string_slice.value = "Hello"

In [2]:
# Test: Inspect Dcel Types
import json

res = dict()
for each in dtest:
    res[each] = { "before": dinspect(dtest[each]) }
    
# Modify the slices so we can see how they modify
# their precursor Dcel's.

dtest_string_slice.value = "Konichiwa"
dtest_file_slice_slice.value = "rouge"

for each in dtest:
    res[each]['after'] = dinspect(dtest[each])
    
for each in res:
    print(each)
    print("--inspect")
    print(res[each]['before'])
    print("--modify")
    print(res[each]['after'])
    
## Restore values before modification
dtest_file.value = "Some are red, some are green."
dtest_string_slice.value = "Hello"


NameError: name 'dtest' is not defined

# Tests: Other

In [51]:
## Test `getinfo()`
from fs import open_fs
a = Dcel('file://fs/fs', service_class=open_fs, writeable=False)
b = a['fruit']
print(b.getinfo(namespaces=['details']).raw)

{'basic': {'name': 'fruit', 'is_dir': True}, 'details': {'_write': ['accessed', 'modified'], 'accessed': 1703334794.5909467, 'modified': 1689681041.4229329, 'size': 4096, 'type': 1, 'created': None, 'metadata_changed': 1689681041.4229329}, 'hosts': ['file://fs/fs']}


In [46]:
c = a[None]
print(c.getinfo().raw)

{'basic': {'name': '', 'is_dir': True}, 'hosts': ['file://fs/fs']}


In [16]:
## Test 'writeable == False'
h = Dcel('hello', writeable=False)
try:
    h.value = 'konichiwa'
except Exception as e:
    print(f'Exception: {e}')

Exception: not writeable


In [5]:
## Test Dcel from Dcel
hi = Dcel(h, writeable=True)
print(hi == h)
print(hi)
hi.value = 'konichiwa'
print(hi)
print(h)

False
hello
konichiwa
hello


In [6]:
## Test subdirectory lookup returns unique Dcel
from fs import open_fs
a = Dcel(address='fs/fs', service_class=open_fs, writeable=False)
assert(a != a['fruit'])

In [7]:
assert(a['fruit'] == a['fruit'])

In [8]:
b = Dcel(a['fruit'], writeable=False)
a['fruit'] == b

True

In [9]:
a['fruit']._writeable

False

In [10]:
b._writeable

False

In [11]:
b.listdir()

['cherries.html.txt', 'apples.txt']

In [12]:
b['cherries.html.txt']._writeable

False

In [13]:
a['fruit']['cherries.html.txt']._writeable

False

In [14]:
b['cherries.html.txt'] == a['fruit']['cherries.html.txt']

True

In [11]:
## Test failed lookup
try:
    a['boric_acid']
except KeyError as e:
    print(f'KeyError: {e}')

KeyError: 'boric_acid'


In [12]:
d = { 'color': 'red' }
[ea for ea in d]

['color']

In [48]:
help(list().__getitem__)

Help on built-in function __getitem__:

__getitem__(...) method of builtins.list instance
    x.__getitem__(y) <==> x[y]



In [56]:
[ea for ea in d.keys()]

['color']

In [63]:
## Testing list 'in' operator
[ ea for ea in a ]

# RESULTS in TypeError
#   trying to use an 'int' where the __getitem__() method wants a key

TypeError: can only concatenate str (not "int") to str

In [64]:
a.keys()

['test', '.cosm', 'fruit', 'tmp_updateMultiValue.txt', 'numbers', 'boats', '@']

In [6]:
from fs import open_fs
from inspect import isclass, isfunction, ismethod

print(isclass(open_fs))
print(isfunction(open_fs))
print(ismethod(open_fs))

print(open_fs)

False
False
True
<bound method Registry.open_fs of <fs-registry ['userdata', 'userconf', 'sitedata', 'siteconf', 'usercache', 'userlog', 'ftp', 'ftps', 'mem', 'file', 'osfs', 'tar', 'temp', 'zip', 'sftp', 'ssh', 's3']>>


In [11]:
## TEST: subscript interface
a = Dcel(address='.',service_class=open_fs, writeable=False)
print(a['fs/fs/fruit/apples.txt'])

TypeError: __init__() got an unexpected keyword argument 'writeable'

In [8]:
from fs.subfs import SubFS

In [9]:
help(SubFS)

Help on class SubFS in module fs.subfs:

class SubFS(fs.wrapfs.WrapFS, typing.Generic)
 |  SubFS(parent_fs, path)
 |  
 |  A sub-directory on a parent filesystem.
 |  
 |  A SubFS is a filesystem object that maps to a sub-directory of
 |  another filesystem. This is the object that is returned by
 |  `~fs.base.FS.opendir`.
 |  
 |  Method resolution order:
 |      SubFS
 |      fs.wrapfs.WrapFS
 |      fs.base.FS
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, parent_fs, path)
 |      Create a filesystem. See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  delegate_fs(self)
 |      Get the proxied filesystem.
 |      
 |      This method should return a filesystem for methods not
 |      associated with a path, e.g. `~fs.base.FS.getmeta`.
 |  
 |  delegate_path(self, path)
 |      Encode a path for proxied filesystem.
 |      
 |   

In [21]:
f.remove('/hello.nurse')

In [18]:
f.writetext('/hello.nurse','hello')

In [22]:
f.listdir('/')

['test', '.cosm', 'fruit', 'tmp_updateMultiValue.txt', 'numbers', 'boats', '@']

In [20]:
f.readtext('/hello.nurse')

'hello'

In [35]:
## TEST: dict() address type
#  REQUIRES: DictFS
from DictFS import DictFS
a = Dcel({'a':'pirate','b':'ship'}, service_class=DictFS)
print(a.listdir('/'))

['a', 'b']


In [41]:
## TEST: nesting other types inside DictFS
from DictFS import DictFS
from fs import open_fs
a = Dcel('.',service_class=open_fs)
b = Dcel({'a':a}, service_class=DictFS)
print(b.listdir('/a'))

[]


In [21]:
### Test service_class=open_fs

from fs import open_fs

a = Dcel(address='file://fs/fs',
         service_class=open_fs
        )

In [22]:
print(a.address)
print(a.service)
print(a.listdir())

/
<osfs '/home/raygan/Cosms/Dboy/Laydbug/dev/cburnfs_py/cburnfs/fs/fs'>
['test', '.cosm', 'fruit', 'tmp_updateMultiValue.txt', 'numbers', 'boats', '@']


In [28]:
print(a.getinfo(namespaces=["details"]).raw)

{'basic': {'name': '', 'is_dir': True}, 'details': {'_write': ['accessed', 'modified'], 'accessed': 1703333622.6908386, 'modified': 1703041388.8880703, 'size': 4096, 'type': 1, 'created': None, 'metadata_changed': 1703041388.8880703}, 'hosts': ['file://fs/fs']}


In [9]:
### TEST HOSTNAME GETTER

from fs.osfs import OSFS
d = Dcel(address='fs/fs/boats',
        service_class=OSFS)

print(d.hostname)

file:///home/raygan/Cosms/Dboy/Laydbug/dev/cburnfs_py/cburnfs/fs/fs/boats/


In [11]:
# Dcel write to slice, and flush to storage.

from fs.osfs import OSFS
d = Dcel(address='fs/fs/boats',
        service_class=OSFS)
print(d["skiff.txt"])
#d["skiff.txt"] = "My skiff is WAY faster than your scull."
print(d["skiff.txt"])
d["skiff.txt"][3:8] = "barge"
print(d["skiff.txt"])
d["skiff.txt"].flush()
#print(d["skiff.txt"])

Dcel::__init__() self.service=<osfs 'fs/fs/boats'>
My barge is WAY faster than your scull.
My barge is WAY faster than your scull.
My barge is WAY faster than your scull.


In [8]:
e = Dcel(address='fs/fs/boats',
        service_class=OSFS)
print(e["skiff.txt"])

My barge is WAY faster than your scull.


In [2]:
# Dcel Referencing vs. Re-binding

# re-binding
a = Dcel("grand opening")
b = Dcel(a)

print(repr(a))
print(repr(b))
assert(a is b)

# referencing
c = ~a
print(repr(c))
print(repr(~c))

somedict = { "one": a,
             "two": c
           }

from DictFS import DictFS
d = Dcel(address=somedict, service_class=DictFS)

print(d.listdir())
print(d['two'])
print(repr(d['two']._value))



['one', 'two']
grand opening


In [4]:
# Dcel slice test.

x = Dcel('once upon a time in the lost land...')
z = x[7:]
print(f"x: {repr(x)} map: {x._map}")
print(f"z: {repr(z)} map: {z._map}")
z.value = 'on a trip to the supermarket...'
t = z[0:2]
t.value = "over"
w = z[3:9]
w.value = "an excapade"
print(f"x: {repr(x)} map: {x._map}")
print(f"z: {repr(z)} map: {z._map}")
print('\n')
v = w[3:5]
v.value = "XXX"
u = w[9:11]
u.value = "Dumpling"

# assert the map references cascade one to the next
print(f"x: {repr(x)} map: {x._map}")
print(f"z: {repr(z)} map: {z._map}")
print(f"w: {repr(w)} map: {w._map}")
print(f"v: {repr(v)} map: {v._map}")

# this should print with no errors
# and the sentence should be "once upon an Xcapade to the supermarket..."
print(x)


x: <__main__.Dcel object at 0x7fed686ec550> map: None
z: <__main__.Dcel object at 0x7fed702da6a0> map: None
x: <__main__.Dcel object at 0x7fed686ec550> map: {7: (None, <__main__.Dcel object at 0x7fed702da6a0>)}
z: <__main__.Dcel object at 0x7fed702da6a0> map: {0: (2, <__main__.Dcel object at 0x7fed68641880>), 3: (9, <__main__.Dcel object at 0x7fed68641c10>)}


x: <__main__.Dcel object at 0x7fed686ec550> map: {7: (None, <__main__.Dcel object at 0x7fed702da6a0>)}
z: <__main__.Dcel object at 0x7fed702da6a0> map: {0: (2, <__main__.Dcel object at 0x7fed68641880>), 3: (9, <__main__.Dcel object at 0x7fed68641c10>)}
w: <__main__.Dcel object at 0x7fed68641c10> map: {3: (5, <__main__.Dcel object at 0x7fed700a6160>), 9: (11, <__main__.Dcel object at 0x7fed7023afa0>)}
v: <__main__.Dcel object at 0x7fed700a6160> map: None
once upover an XXXcapaDumpling to the supermarket...


In [5]:
print(x)

once upover an XXXcapaDumpling to the supermarket...


In [12]:

#z.value = 'on a trip to the supermarket...'

print(z)
print(x)

print(w)
print(f"x: {x}")
print(x._map)
print(z._map)
print(w._map)
print("\n")
print(x.address)

over an XXXcapaDumpling to the supermarket...
once upover an XXXcapaDumpling to the supermarket...
an XXXcapaDumpling
x: once upover an XXXcapaDumpling to the supermarket...
{7: (None, <__main__.Dcel object at 0x7fc168355cd0>)}
{0: (2, <__main__.Dcel object at 0x7fc151f15940>), 3: (9, <__main__.Dcel object at 0x7fc152080ac0>)}
{3: (5, <__main__.Dcel object at 0x7fc1681f1490>), 9: (11, <__main__.Dcel object at 0x7fc152080df0>)}




AttributeError: 'Dcel' object has no attribute 'address'

In [13]:
# Test Dcel Reference Inversion

d = Dcel("hello")
dref = ~d
drefd = ~dref

print(f"{repr(d)}")
print(f"{repr(dref)}")
print(f"{repr(drefd)}")

assert(d is drefd)

In [14]:
# Test 'in' operator with file system backing.

from fs.osfs import OSFS
d = Dcel(address='fs/fs/boats',
        service_class=OSFS)

for item in d:
    print(item)

TypeError: None

In [5]:
# Test 'in' operator with str initialization.
# The Dcel::__iter__() method should catch this.
a = Dcel("hello")
for each in a:
    print(each)

h
e
l
l
o


In [14]:
# Test 'in' operator with list initialization.

a = Dcel([1,2,3])
for i in a:
    print(i)

1
2
3


In [99]:
# for JSONEncoder see DcelJSONEncoder notebook.

In [25]:
# bytes encoder playground

a = "123".encode("utf-8")
b = bytes("456".encode("utf-8"))
a += b
print(a.decode('utf-8'))
print(len(a))

123456
6


In [62]:
# Test Dcel with formula and MulticelSeqFS

from fs.osfs import OSFS
from MulticelSeqFS import MulticelSeqFS

def globstar(a):
    items = [a[ea] for ea in a.listdir()]
    return items

a = Dcel(address='fs/fs', 
         service_class=OSFS)
b = Dcel(formula=globstar, args=[~a])
c = Dcel(address=b,
         service_class=MulticelSeqFS)

print(c.listdir())
print(c['etc'].listdir())

['var', 'hosts', 'types', 'etc', 'var', 'hosts', 'types', 'etc', 'var', 'hosts', 'types', 'etc', 'cherries.html.txt', 'apples.txt', 'skiff.txt', 'junk.txt', 'catamaran.txt', 'var', 'hosts', 'types', 'etc']
['ssh', 'fstab', 'ssh', 'fstab', 'ssh', 'fstab', 'ssh', 'fstab']


In [60]:
from fs.osfs import OSFS

q = Dcel(address='fs/fs', 
         service_class=OSFS
        )
print(q.isdir())
print(q.value)
print(q['.cosm/etc/fstab'].value)
#print(q['.cosm/etc/fstab'].readbytes('.'))

True

# # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths relative to the fstab's location, ie: for ./.cosm/etc/fstab the relative root is ../../../



file://fs2.localhost/  {cburnuser}/example/  cburnfs user,shortid=f2,idcard=localuser 0 0
file://fs.localhost    {cburnuser}/example/  cburnfs user,shortid=fs,idcard=localuser 0 0
file://fs3.localhost/  {cburnuser}/example/  cburnfs user,shortid=f3,idcard=localuser 0 0
file://cburnwebui.localhost/  {cburnuser}/example/  cburnf

In [6]:
from DictFS import DictFS

a = Dcel(address={'one':'1'}, 
         service_class=DictFS
        )
print(a.isdir())
print(a.value)
print(a['one'].value)

True
{'one': '1'}
1


In [7]:
b = Dcel(address=a, 
         service_class=DictFS
        )

In [8]:
type(a.value) #'one',namespaces=['dcel']).raw

dict

In [7]:
print(type(b.address))
print(b.address)
print(type(b.value))
print(type(b._value))
print(b.service)

<class 'str'>
/
<class 'dict'>
<class 'dict'>


In [8]:
{1:1}.items()

dict_items([(1, 1)])

In [10]:
def addem(a,b,msg=None):
    if msg:
        print(msg)
    return a+b

b = Dcel(value=2)
c = Dcel(value=1)
d = Dcel(formula=addem,args=[c,b])
e = Dcel(formula=addem,args=[d,d])
f = Dcel(formula=addem,args=[d,e],kwargs={'msg':'hello'})

print(e.value)
b.args = [3]
print(f.value)

6
hello
9


In [17]:
def say_hi():
    return("hi there.")

g = Dcel(formula=say_hi, args=[])

print(g.value)

hi there.


In [5]:
def argumentor(x):
    return x

class dc:
    def __init__(self, dcel):
        self._dcel = dcel
    
bb = ~b
type(argumentor(~bb))

__main__.Dcel

In [6]:
g = Dcel(formula=argumentor, 
         args=[b])
g.value

2

In [7]:
from fs.osfs import OSFS
from DictFS import DictFS

d = Dcel(service_class=OSFS, 
         address='fs'
        )

d2 = Dcel(service_class=DictFS, 
         address={'fruit': {
                'cherries.html.txt': '<b>Sakuranbo</b>'
                    },
                   'boats':'skiff'
                  }
        )

In [8]:
a = d2.path_lookup('fruit/cherries.html.txt')
a.value

'<b>Sakuranbo</b>'

In [9]:
d2['fruit']['cherries.html.txt'].value

'<b>Sakuranbo</b>'

In [10]:
d2['fruit']['cherries.html.txt'].value = 'hello'

In [14]:
d['fruit']['apples.txt'].value = 'red'

In [7]:
d['fruit']['apples.txt'].value

'red'

In [8]:
for ea in d,d2:
    print(ea['fruit'].getinfo().raw)
    print(ea['fruit'].listdir())
    print(ea['fruit']['cherries.html.txt'].value)

{'basic': {'name': 'fruit', 'is_dir': True}, 'dcel': {'hosts': ['localhost']}}
['apples.txt', 'cherries.html.txt']
<i>sakuranbo</i> means <b>cherries</b>

{'basic': {'name': 'fruit', 'is_dir': True}, 'dcel': {'hosts': ['localhost']}}
['cherries.html.txt']
hello


In [9]:
for ea in d,d2:
    print(ea.getinfo().raw)
    print(ea.listdir())
    print(ea.value)



{'basic': {'name': '', 'is_dir': True}, 'dcel': {'hosts': ['localhost']}}
['fruit', 'boats', '.cosm', 'test.txt', '.git']
None
{'basic': {'name': '', 'is_dir': True}, 'dcel': {'hosts': ['localhost']}}
['fruit', 'boats']
{'fruit': {'cherries.html.txt': 'hello'}, 'boats': 'skiff'}


In [14]:
from urllib.parse import urlparse
from fs.opener import parse

urls = ['sshfs://raygan.com:80/path',
       'http://localhost:80/path;parameters?query=argument#fragment',
       'file://localhost/path',
        'fs'
       ]

for u in urls:
    print('---')
    parsed = urlparse(u).netloc
    print('\''+parsed+'\'')
    fsparsed = parse(u)
    print(fsparsed)
    


---
'raygan.com:80'
ParseResult(protocol='sshfs', username=None, password=None, resource='raygan.com:80/path', params={}, path=None)
---
'localhost:80'
ParseResult(protocol='http', username=None, password=None, resource='localhost:80/path;parameters', params={'query': 'argument#fragment'}, path=None)
---
'localhost'
ParseResult(protocol='file', username=None, password=None, resource='localhost/path', params={}, path=None)
---
''


ParseError: 'fs' is not a fs2 url

In [3]:
class Dtest3:
    def __init__(self):
        self.address = 'blue'
    def getinfo(self,addr=None):
        if addr is None:
            addr = self.address
        print(addr)

a = Dtest3()
a.getinfo()

blue


In [2]:
class Dtest2:
    def __init__(self):
        pass
    
    @property
    def value(self):
        try: 
            return self._value
        except:
            print('no dice')
            return ''

In [4]:
a = Dtest2()
a.value

no dice


''

In [16]:
# Init Logic

class Dtest:
    def __init__(self,
                 address = None,
                 service = None,
                 args = None,
                 formula = None,
                 value = None,
                 service_class = None,
                 service_args = None,
                 # obsolete
                 hostname = 'example',
                ):
        if service and not address is None:
            print('addr-service pair')
        elif service_class and not address is None:
            print('Create new service object')
            if service_args:
                print('service args applied')
        if formula and args:
            print('formula-args pair')
        elif formula:
            print('formula without args')
        if not value is None:
            print('value explicitly set')
            self._value = value
        

In [18]:
a = Dtest(
          service='one',
          address='two',
          formula='three',
          args=[1,2,3,4],
          service_args=[1,2,3],
         )

b = Dtest(
          service_class='one',
          service_args=[1,2,3],
          address='two',
          formula='three',
          value='',
         )


addr-service pair
formula-args pair
Create new service object
service args applied
formula without args
value explicitly set


In [4]:
d = Dcel()
type(Dcel)


abc.ABCMeta

In [3]:
x = None
try:
    y = x[1]
except TypeError:
    x = dict()
    y = x[1] = 'hello'

except KeyError:
    y = x[1] = 'hello'

print(y)

hello


In [10]:
from DictFS import DictFS

fsdict = {".cosm":
          {"etc":
           {"fstab": "fs.raygan.com / sshfs"
           }
          }
         }

d = Dcel(fsdict,DictFS)



In [5]:


if str(type(DictFS)) == "abc.ABCMeta":
    print('OK')
    
print(str(type(DictFS)))

<class 'abc.ABCMeta'>


In [13]:
from FileService import FileService
from fs.osfs import OSFS

d = Dcel('fs',OSFS)

In [14]:
d.address
type(OSFS)

abc.ABCMeta

In [5]:
d.getinfo('.cosm/etc/').raw

AttributeError: 'Dcel' object has no attribute 'service'

In [6]:
d['.cosm']['etc'].listdir()

getitem...
getitem: key: .cosm
getitem: path: /.cosm
getitem...
getitem: key: etc
getitem: path: /.cosm/etc


['fstab']